# 의사결정트리 분류 복잡도 최적화

## #01. 준비작업

### [1] 패키지 가져오기

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Intel SKlearn 하드웨어 가속 패치 설정
import sys
if sys.platform == 'win32':
    from sklearnex import patch_sklearn
    patch_sklearn()

from helper.util import *
from helper.plot import *
from helper.analysis import *
from helper.classification import *

# 분석 알고리즘
from sklearn.tree import DecisionTreeClassifier, plot_tree

### [2] 데이터 가져오기

In [ ]:
origin = my_read_excel("https://data.hossam.kr/mldata/breast-cancer-wisconsin.xlsx", index_col="id", categories=["diagnosis"], info=False)

### [3] 전처리

스케일링 없이 `7:3` 비율로 데이터 분할

In [ ]:
df1 = my_labelling(origin, 'diagnosis')
x_train, x_test, y_train, y_test = my_train_test_split(df1, 'diagnosis', test_size=0.3)

## #02. 기본 분류 모형

In [ ]:
cv_estimator = my_dtree_classification(x_train, y_train, x_test, y_test, hist=False, roc=False, pr=False, conf_matrix=False, learning_curve=False)

## #03. Pruning (가지치기)

dtree 분류기 객체의 `cost_complexity_pruning_path()`메서드는 각 과정마다 효과적인 불순도와 이를 위한 알파값을 반환한다.

### [1] 알파값, 불순도 구하기

In [ ]:
dtree = DecisionTreeClassifier(random_state=1234)
path = dtree.cost_complexity_pruning_path(x_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities
print(ccp_alphas, impurities)

### [2] 알파값, 불순도 시각화

In [ ]:
fig = plt.figure(figsize=(10, 5), dpi=200)
ax = fig.gca()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker="o", drawstyle="steps-post")
ax.set_xlabel("ccp alpha")
ax.set_ylabel("불순도")
ax.grid()
plt.show()
plt.close()

> 알파값이 커질 수록 불순도가 높아진다. 그러므로 최적의 정확도와 가장 낮은 알파값을 동시에 충족하는 모형을 찾아야 한다.

### [3] 알파값을 적용한 DTree 객체 생성후 노드의 수 확인

In [ ]:
# 의미 없는 양 끝의 값을 제외하고 알파값 목록을 생성
ccp_alphas = ccp_alphas[1:-1]
estimator_list = []

for ccp_alpha in ccp_alphas:
    dtree = DecisionTreeClassifier(random_state=1234, ccp_alpha=ccp_alpha)
    dtree.fit(x_train, y_train)

    # 분류기 객체를 리스트에 담는다.
    estimator_list.append(dtree)

# 각 분류기에 따른 트리의 노드 수 확인
t_nodes = [e.tree_.node_count for e in estimator_list]
t_nodes

> 알파값에 의해 노드 수가 점차 줄어들고 있다. 노드 수가 지나치게 작을 경우 정확도는 분명 떨어질 것이다.

### [4] 알파값에 따른 의사결정트리 비교

In [ ]:
for i, v in enumerate(estimator_list):
    print("[node_size: {0}, ccp_alpha: {1:.4f}]".format(v.tree_.node_count, ccp_alphas[i]), "="*50, "\n")
    my_classification_result(v, x_test, y_test, hist=False, roc=False, pr=False, conf_matrix=False, learning_curve=False)
    print("\n" * 10)

### [5] 알파값에 따른 성능 변화

In [ ]:
train_scores = []
test_scores = []

for v in estimator_list:
    train_scores.append(v.score(x_train, y_train))
    test_scores.append(v.score(x_test, y_test))

fig = plt.figure(figsize=(10, 5), dpi=200)
ax = fig.gca()
ax.set_xlabel("Alpha")
ax.set_ylabel("Accuracy")
ax.set_title("Alpha vs. Accuracy for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker="o", label="train set accuacy", drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker="o", label="test set accuracy", drawstyle="steps-post")
ax.legend()
ax.grid()
plt.show()
plt.close()

### [6] 검증데이터에 대한 스코어에 따른 알파값

In [ ]:
df = DataFrame({"test acc": test_scores, "alpha":ccp_alphas})
df.sort_values(["test acc", "alpha"], ascending=[False, True], inplace=True)
df.reset_index(drop=True, inplace=True)
my_pretty_table(df)

In [ ]:
"최적의 알파값:", df['alpha'][0]

### [7] 알파값을 적용한 새로운 분석 모형

In [ ]:
final_estimator = DecisionTreeClassifier(criterion='gini', random_state=1234, ccp_alpha=df['alpha'][0])
final_estimator.fit(x_train, y_train)
my_tree(final_estimator)

### [8] 알파값 리스트를 적용한 RandomizeSearhCV 기능 확인

> 모듈화 내용

In [ ]:
my_dtree_classification(x_train, y_train, x_test, y_test)